### LSTM

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])
data

,Close,sma,ema20,ema50,ema200,rsi,macd,macd_signal,Close_future,Price_Up
Date,,,,,,,,,,
2015-04-04,253.697006,266.058734,255.661524,258.734938,307.338692,47.508324,-4.193133,-3.906538,260.597992,1
2015-04-05,260.597992,265.654567,256.131663,258.807999,306.873610,52.432292,-3.252122,-3.775655,255.492004,0
2015-04-06,255.492004,264.962268,256.070744,258.677960,306.362350,48.785829,-2.885117,-3.597547,253.179993,0
2015-04-07,253.179993,264.256468,255.795434,258.462354,305.833173,47.185618,-2.749132,-3.427864,245.022003,0
2015-04-08,245.022003,262.770301,254.769393,257.935281,305.228086,41.956153,-3.262042,-3.394700,243.675995,0
...,...,...,...,...,...,...,...,...,...,...
2024-11-28,95652.468750,84858.846094,90477.864743,81038.222170,67603.340183,65.862719,5520.472967,6077.440370,97461.523438,1
2024-11-29,97461.523438,85696.245573,91142.975095,81682.273200,67900.436534,68.175702,5415.826234,5945.117543,96449.054688,0
2024-11-30,96449.054688,86570.707812,91648.316009,82261.362670,68184.502386,65.500818,5191.352498,5794.364534,97279.789062,1


In [23]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target].to_numpy()


In [24]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [25]:
import numpy as np

sequence_length = 60

X_sequences = []
y_sequences = []

for i in range(sequence_length, len(X_scaled)):
    X_sequences.append(X_scaled[i-sequence_length:i])
    y_sequences.append(y[i])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.01, shuffle=False
)

In [29]:
# Model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\ivoto\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.5263 - loss: 0.6934 - val_accuracy: 0.4857 - val_loss: 0.7000
Epoch 2/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5291 - loss: 0.6883 - val_accuracy: 0.4857 - val_loss: 0.6947
Epoch 3/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.5176 - loss: 0.6909 - val_accuracy: 0.4857 - val_loss: 0.7050
Epoch 4/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.5338 - loss: 0.6889 - val_accuracy: 0.5143 - val_loss: 0.6928
Epoch 5/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.5479 - loss: 0.6861 - val_accuracy: 0.4286 - val_loss: 0.6952
Epoch 6/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.5244 - loss: 0.6893 - val_accuracy: 0.4857 - val_loss: 0.6952
Epoch 7/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.5372 - loss: 0.6876 - val_accuracy: 0.4857 - val_loss: 0.6978
Epoch 8/20
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.5216 - loss: 0.6879 - val_accu

In [31]:
# 9. Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Precisión del modelo LSTM: {accuracy:.2f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4315 - loss: 0.6934
Precisión del modelo LSTM: 0.43
